# **4.4 Orthonormal Bases and Gram-Schmidt**

- The vectors $q_1, \dots, q_n$ are **orthogonal** when their dot products $q_i \cdot q_j$ are zero. More precisely: $q_i^T q_j = 0$ whenever $i \neq j$.  
  By dividing each vector by its length, the vectors become unit vectors. Together, they form an **orthonormal basis**.

**Definition:** The vectors $q_1, \dots, q_n$ are orthonormal if  
$$
q_i^T q_j =
\begin{cases}
0 & \text{if } i \neq j \text{ (orthogonal)} \\
1 & \text{if } i = j \text{ (unit length)}
\end{cases}
$$

- A matrix with orthonormal columns is denoted $Q$.
- $Q$ is easy to work with because  
$$
Q^T Q = I
$$  
  meaning its columns are orthonormal. $Q$ does **not** have to be square.

- If $Q$ is square, $Q^T Q = I$ implies $Q^T = Q^{-1}$. Such a square orthonormal matrix is called an **orthogonal matrix**.

- **Reflections and Rotations:**  
  Operations like reflections, rotations, permutations, and multiplication by orthogonal matrices preserve lengths and angles:  
  $$
  \|Qx\| = \|x\|, \quad (Qx)^T (Qy) = x^T y
  $$

### **Projections Using Orthonormal Bases**

- With an orthonormal basis $Q$, the least squares formulas simplify:  
$$
\hat{x} = Q^T b, \quad p = Q \hat{x}, \quad P = QQ^T
$$

- No matrices need to be inverted; projections are just dot products:  
$$
p = q_1 (q_1^T b) + \dots + q_n (q_n^T b)
$$

- If $Q$ is square ($m=n$), then $QQ^T = I$, and the projection of $b$ onto the whole space is $b$ itself:  
$$
b = QQ^T b = q_1(q_1^T b) + \dots + q_n(q_n^T b)
$$

- This is the foundation for Fourier series and other transforms: vectors/functions are decomposed into perpendicular components and can be recombined.

In [1]:
import numpy as np

def project_onto_orthonormal(Q, b):
    """
    Project vector b onto the subspace spanned by the orthonormal columns of Q.
    
    Q: (m × k) matrix with orthonormal columns (Q^T Q = I)
    b: (m,) vector

    Returns:
        p: projection of b onto Col(Q)
        e: residual vector b - p
    """
    Q = np.asarray(Q, dtype=float)
    b = np.asarray(b, dtype=float)

    p = Q @ (Q.T @ b)
    e = b - p
    return p, e

import numpy as np

# orthonormal basis (2D subspace in R^3)
Q = np.array([
    [1/np.sqrt(2),  1/np.sqrt(6)],
    [1/np.sqrt(2), -1/np.sqrt(6)],
    [0,             2/np.sqrt(6)]
])

# sample vector to project
b = np.array([3.0, 1.0, 2.0])

p, e = project_onto_orthonormal(Q, b)

print("projection p:", p)
print("residual e:", e)

projection p: [3. 1. 2.]
residual e: [ 0.00000000e+00  5.55111512e-16 -4.44089210e-16]


### **The Gram-Schmidt Process**

- Converts a set of independent vectors $a, b, c$ into orthonormal vectors $q_1, q_2, q_3$:
  1. $A = a$ (first vector)
  2. $B = b - \frac{A^T b}{A^T A} A$ (subtract projection onto $A$)
  3. $C = c - \frac{A^T c}{A^T A} A - \frac{B^T c}{B^T B} B$ (subtract projections onto $A$ and $B$)
- Normalize to get orthonormal vectors:  
$$
q_1 = \frac{A}{\|A\|}, \quad q_2 = \frac{B}{\|B\|}, \quad q_3 = \frac{C}{\|C\|}
$$`

In [2]:
import numpy as np

def gram_schmidt(A):
    """
    Classical Gram-Schmidt process.
    Input:
        A: (m × n) matrix with independent columns
    Output:
        Q: (m × n) with orthonormal columns
    """
    A = np.asarray(A, dtype=float)
    m, n = A.shape
    Q = np.zeros((m, n))

    for j in range(n):
        v = A[:, j]

        # subtract projections onto previous q_i
        for i in range(j):
            qi = Q[:, i]
            v = v - (qi @ v) * qi

        # normalize
        Q[:, j] = v / np.linalg.norm(v)

    return Q

A = np.array([
    [1.0, 1.0, 1.0],
    [1.0, 2.0, 3.0],
    [1.0, 4.0, 9.0]
])

Q = gram_schmidt(A)

print("Q:\n", Q)
print("Q^T Q:\n", Q.T @ Q)   # should be ~ identity

Q:
 [[ 0.57735027 -0.6172134   0.53452248]
 [ 0.57735027 -0.15430335 -0.80178373]
 [ 0.57735027  0.77151675  0.26726124]]
Q^T Q:
 [[ 1.00000000e+00 -8.78595614e-16  4.26800386e-15]
 [-8.78595614e-16  1.00000000e+00 -2.34491098e-15]
 [ 4.26800386e-15 -2.34491098e-15  1.00000000e+00]]


### **The Factorization $A = QR$**

- Start with $A = [a\ b\ c]$ and end with $Q = [q_1\ q_2\ q_3]$. There exists an upper triangular matrix $R$ such that:  
$$
A = QR, \quad
R =
\begin{bmatrix}
q_1^T a & q_1^T b & q_1^T c \\
0 & q_2^T b & q_2^T c \\
0 & 0 & q_3^T c
\end{bmatrix}
$$

- Any $m \times n$ matrix $A$ with independent columns can be factorized as $A = QR$ with $Q$ orthonormal and $R$ upper triangular with positive diagonal.

- **Least Squares Application:**  
$$
A^T A \hat{x} = A^T b \quad \Rightarrow \quad R^T R \hat{x} = R^T Q^T b \quad \Rightarrow \quad R \hat{x} = Q^T b \quad \Rightarrow \quad \hat{x} = R^{-1} Q^T b
$$  
Back substitution is fast and avoids matrix inversion.

In [3]:
import numpy as np

def qr_factorization(A):
    """
    Computes the QR factorization A = QR using Classical Gram-Schmidt.
    
    A: (m × n) with independent columns
    Returns:
        Q: (m × n) with orthonormal columns
        R: (n × n) upper triangular
    """
    A = np.asarray(A, dtype=float)
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j].copy()

        # subtract projections onto previous q_i
        for i in range(j):
            R[i, j] = Q[:, i] @ A[:, j]
            v = v - R[i, j] * Q[:, i]

        # normalize
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]

    return Q, R

A = np.array([
    [1.0, 1.0, 1.0],
    [1.0, 2.0, 3.0],
    [1.0, 4.0, 9.0]
])

Q, R = qr_factorization(A)

print("Q:\n", Q)
print("R:\n", R)
print("Check A ≈ QR:\n", Q @ R)

Q:
 [[ 0.57735027 -0.6172134   0.53452248]
 [ 0.57735027 -0.15430335 -0.80178373]
 [ 0.57735027  0.77151675  0.26726124]]
R:
 [[1.73205081 4.04145188 7.5055535 ]
 [0.         2.1602469  5.8635273 ]
 [0.         0.         0.53452248]]
Check A ≈ QR:
 [[1. 1. 1.]
 [1. 2. 3.]
 [1. 4. 9.]]


**Key Ideas**

1. For orthonormal $q_1, \dots, q_n$ forming $Q$:  
   $$
   Q^T Q = I
   $$

2. If $Q$ is square (orthogonal), $Q^T = Q^{-1}$.

3. Length preservation: $\|Qx\| = \|x\|$.

4. Projection onto the column space of $Q$: $P = QQ^T$.

5. If $Q$ is square:  
$$
b = QQ^T b = q_1 (q_1^T b) + \dots + q_n (q_n^T b)
$$

6. Gram-Schmidt produces orthonormal vectors and the factorization $A = QR = (\text{orthogonal } Q)(\text{triangular } R)$.